In [ ]:
#  filter out known/unknown results according to short answers only based on no search results
datasets=["asqa", "nq", "trivia-qa", "hotpot-qa"]
split=('train')
chat_model="llama7b"
search_engine="kiltbm25"
rerank_model="e5base"


import os
import sys
sys.path.append("../")
import json
import string
import re

def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""

  def remove_articles(text):
    return re.sub(r'\b(a|an|the)\b', ' ', text)

  def white_space_fix(text):
    return ' '.join(text.split())

  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)

  def lower(text):
    return text.lower()

  return white_space_fix(remove_articles(remove_punc(lower(s))))

def calculate_short_answer_EM(generated_answer, gold_answers):
    generated_answer=normalize_answer(generated_answer)
    match=0
    for gold_answer in gold_answers:
        gold_answer=normalize_answer(gold_answer)
        if gold_answer in generated_answer:
            match+=1
    return {
        "recall": match/len(gold_answers),
        "exact_match": match>0
    }
      
for dataset in datasets:
    # print(f"dataset: {dataset}, split: {split}, chat_model: {chat_model}, search_engine: {search_engine}, rerank_model: {rerank_model}")
    #  no search chat results
    if split == "trainmore":
        without_search_chat_results_file=f'../user_intent_data/{dataset}/{chat_model}/without_search/{chat_model}-{dataset}-train.jsonl'
        without_search_chat_results_file2=f'../user_intent_data/{dataset}/{chat_model}/without_search/{chat_model}-{dataset}-trainmore.jsonl'
        if os.path.exists(without_search_chat_results_file2):
            without_search_chat_results=[json.loads(line) for line in open(without_search_chat_results_file, "r", encoding="utf-8")]
            without_search_chat_results.extend([json.loads(line) for line in open(without_search_chat_results_file2, "r", encoding="utf-8")])
        else:
            without_search_chat_results=[json.loads(line) for line in open(without_search_chat_results_file, "r", encoding="utf-8")]
    else:
        without_search_chat_results_file=f'../user_intent_data/{dataset}/{chat_model}/without_search/{chat_model}-{dataset}-{split}.jsonl'
        without_search_chat_results=[json.loads(line) for line in open(without_search_chat_results_file, "r", encoding="utf-8")]
    generated_answers=[data_line[f"{chat_model}_without_search_answer"] for data_line in without_search_chat_results]
    
    if len(without_search_chat_results) == 0:
        continue
    if "short_answers" in without_search_chat_results[0]:
        gold_answers=[data_line["short_answers"] for data_line in without_search_chat_results]
    else:
        if isinstance(without_search_chat_results[0]["answer"], list):
            gold_answers=[data_line["answer"] for data_line in without_search_chat_results]
        else:
            gold_answers=[[data_line["answer"]] for data_line in without_search_chat_results]
    
    without_search_em_results = [calculate_short_answer_EM(generated_answers[idx], gold_answers[idx]) for idx in range(len(generated_answers))]
    
        
    filtered_gpt4_rewrite=[]
    search_tags=[]
    sample_count=0

    thresholds={
        "asqa": 0.5,
        "qampari": 0.7,
        "nq": 0.5,
        "trivia-qa": 0.5,
        "2wiki": 0.5,
        "musique": 0.5,
        "hotpot-qa": 0.5,
    }
    threshold=thresholds[dataset]
    for idx in range(len(without_search_em_results)):
        without_search_chat_results[idx]["dataset"]=dataset
        if without_search_em_results[idx]["recall"] > threshold:
            search_tag="<known> "
            without_search_chat_results[idx]["known"] = True
            sample_count+=1
        else:
            search_tag="<unknown> "
            without_search_chat_results[idx]["known"] = False
        # if gpt4_rewrite_em_results[idx]["recall"] < vanilla_search_em_results[idx]["recall"]:
        #     search_tag+="<original>"
        #     without_search_chat_data[idx]["rewrite"] = False
        # else:
        #     search_tag+="<rewrite>"
        #     without_search_chat_data[idx]["rewrite"] = True
        #  drop some samples if recall are all equal
        # if without_search_em_results[idx]["recall"] == vanilla_search_em_results[idx]["recall"] and without_search_em_results[idx]["recall"] == gpt4_rewrite_em_results[idx]["recall"]:
        #     continue
        # unknow samples are no more than known samples
        if "unknown" in search_tag:
            if sample_count > 0:
                sample_count-=1
            else:
                continue
            
        search_tags.append(search_tag)
        # without_search_chat_data[idx]["search_tags"] = search_tag
        filtered_gpt4_rewrite.append(without_search_chat_results[idx])
        #  filtered sample should be no more than 4000
        # filtered_gpt4_rewrite=filtered_gpt4_rewrite[:5000]
        #  make sure there are no duplicated ids
        filtered_ids=[data_line["id"] for data_line in filtered_gpt4_rewrite]
        assert len(filtered_ids) == len(set(filtered_ids))
        
    print(f"filtered {len(filtered_gpt4_rewrite)} data, total {len(without_search_chat_results)} data")
    print(f"dataset: {dataset}, split: {split}, chat_model: {chat_model}, search_engine: {search_engine}, rerank_model: {rerank_model}")
    search_tag_num_dict={k: search_tags.count(k) for k in set(search_tags)}
    print(search_tag_num_dict)
    print(f"dropped {len(without_search_chat_results)-len(filtered_gpt4_rewrite)} samples")
    if not os.path.exists(f"../user_intent_data/mixed/{dataset}/{chat_model}/{search_engine}/"):
        os.makedirs(f"../user_intent_data/mixed/{dataset}/{chat_model}/{search_engine}/", exist_ok=True)
    #  save filtered results to mixed dataset
    mixed_dataset_file=f"../user_intent_data/mixed/{dataset}/{chat_model}/{search_engine}/filtered-{dataset}-{split}.jsonl"
    with open(mixed_dataset_file, "w", encoding="utf-8") as f:
        for idx in range(len(filtered_gpt4_rewrite)):
            f.write(json.dumps(filtered_gpt4_rewrite[idx], ensure_ascii=False) + "\n")

In [ ]:
#  filter training data for SKR
import json
import os
import random
import string
import re
random.seed(4545)

datasets=("asqa", "nq", "trivia-qa", "musique")
split="trainfew"
chat_model="qwen72b"
search_engine="kiltbm25"
rerank_model="e5base"

def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""

  def remove_articles(text):
    return re.sub(r'\b(a|an|the)\b', ' ', text)

  def white_space_fix(text):
    return ' '.join(text.split())

  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)

  def lower(text):
    return text.lower()

  return white_space_fix(remove_articles(remove_punc(lower(s))))

def calculate_short_answer_EM(generated_answer, gold_answers):
    generated_answer=normalize_answer(generated_answer)
    match=0
    for gold_answer in gold_answers:
        gold_answer=normalize_answer(gold_answer)
        if gold_answer in generated_answer:
            match+=1
    return {
        "recall": match/len(gold_answers),
        "exact_match": match>0
    }

for dataset in datasets:
    no_search_chat_results_file=f'../user_intent_data/{dataset}/{chat_model}/without_search/{chat_model}-{dataset}-{split}.jsonl'
    no_search_chat_results=[json.loads(line) for line in open(no_search_chat_results_file, "r", encoding="utf-8")]
    if "short_answers" in no_search_chat_results[0]:
        gold_answers=[data_line["short_answers"] for data_line in no_search_chat_results]
    else:
        if isinstance(no_search_chat_results[0]["answer"], list):
            gold_answers=[data_line["answer"] for data_line in no_search_chat_results]
        else:
            gold_answers=[[data_line["answer"]] for data_line in no_search_chat_results]
            
    no_search_generated_answers=[data_line[f"{chat_model}_without_search_answer"] for data_line in no_search_chat_results]
    no_search_em_results = [calculate_short_answer_EM(no_search_generated_answers[idx], gold_answers[idx]) for idx in range(len(no_search_generated_answers))]
    
    vanilla_search_chat_results_file=f'../user_intent_data/{dataset}/{chat_model}/{search_engine}/vanilla_search/{rerank_model}-{chat_model}-{dataset}-{split}.jsonl'
    vanilla_search_chat_results=[json.loads(line) for line in open(vanilla_search_chat_results_file, "r", encoding="utf-8")]
    vanilla_search_generated_answers=[data_line[f"{chat_model}_vanilla_search_answer"] for data_line in vanilla_search_chat_results]
    vanilla_search_em_results = [calculate_short_answer_EM(vanilla_search_generated_answers[idx], gold_answers[idx]) for idx in range(len(vanilla_search_generated_answers))]
    
    #  filter out known/unknown results according to short answers
    #  if no search results >= vanilla search results, then it is known
    #  if no search results < vanilla search results, then it is unknown
    filtered_gpt4_rewrite=[]
    search_tags=[]
    sample_count=0
    if "recall" in no_search_em_results[0]:
        for idx in range(len(no_search_em_results)):
            if no_search_em_results[idx]["recall"] > vanilla_search_em_results[idx]["recall"]:
                search_tag="<known> "
                no_search_chat_results[idx]["known"] = True
                sample_count+=1
            elif no_search_em_results[idx]["recall"] < vanilla_search_em_results[idx]["recall"]:
                search_tag="<unknown> "
                no_search_chat_results[idx]["known"] = False
            else:
                continue
            # if both recall are 0, then drop the sample
            if no_search_em_results[idx]["recall"] == 0 and vanilla_search_em_results[idx]["recall"] == 0:
                continue
            filtered_gpt4_rewrite.append(no_search_chat_results[idx])
            search_tags.append(search_tag)
    elif "rouge" in no_search_em_results[0]:
        for idx in range(len(no_search_em_results)):
            if no_search_em_results[idx]["rouge"]["rougeL"] > vanilla_search_em_results[idx]["rouge"]["rougeL"]:
                search_tag="<known> "
                no_search_chat_results[idx]["known"] = True
                sample_count+=1
            elif no_search_em_results[idx]["rouge"]["rougeL"] < vanilla_search_em_results[idx]["rouge"]["rougeL"]:
                search_tag="<unknown> "
                no_search_chat_results[idx]["known"] = False
            else:
                continue
    print(f"filtered {len(filtered_gpt4_rewrite)} data")
    print(f"dataset: {dataset}, split: {split}, chat_model: {chat_model}, search_engine: {search_engine}, rerank_model: {rerank_model}")
    search_tag_num_dict={k: search_tags.count(k) for k in set(search_tags)}
    print(search_tag_num_dict)
    if not os.path.exists(f"../user_intent_data/mixed/{dataset}/{chat_model}/{search_engine}/"):
        os.makedirs(f"../user_intent_data/mixed/{dataset}/{chat_model}/{search_engine}/", exist_ok=True)
    #  save filtered results to mixed dataset
    mixed_dataset_file=f"../user_intent_data/mixed/{dataset}/{chat_model}/{search_engine}/filtered-{dataset}-{split}.jsonl"
    with open(mixed_dataset_file, "w", encoding="utf-8") as f:
        for idx in range(len(filtered_gpt4_rewrite)):
            f.write(json.dumps(filtered_gpt4_rewrite[idx], ensure_ascii=False) + "\n")
    


In [ ]:
#  parse llama question rewrite & separate claim results
import json
datasets=("asqa", "nq", "trivia-qa", "musique", "eli5")
split="test"
chat_model="baichuan7b"
sep_model="v0118"

for dataset in datasets:
    input_file=f"../user_intent_data/{dataset}/rewrite/{sep_model}{chat_model}/unparsed-{sep_model}{chat_model}-{dataset}-{split}.jsonl"
    output_file=f"../user_intent_data/{dataset}/rewrite/{sep_model}{chat_model}/{sep_model}{chat_model}-{dataset}-{split}.jsonl"
    
    with open(input_file, "r") as f:
        data_lines=[json.loads(line) for line in f.readlines()]
        
    for data_line in data_lines:
        output_str = data_line[f"{sep_model}{chat_model}_rewrite"]
        #  parse task
        try:
            task=output_str.split("<Task(")[1].split(")>")[0]
        except:
            print(f"parse timeliness error: {output_str}")
            task=""
        #  parse timeliness
        try:
            timeliness=output_str.split("<Timeliness(")[1].split(")>")[0]
            timeliness=True if timeliness=="True" else False
        except:
            print(f"parse timeliness error: {output_str}")
            timeliness=False
            
        #  parse questions
        try:
            questions=output_str.split("<Questions>")[1].split("</Questions>")[0]
            questions=[q for q in questions.split("<Question(") if q.strip()]
            rewrite_questions=[]
    
            for question_block in questions:
                question=question_block.split(")>")[0]
                need_search=question_block.split("<NeedSearch(")[1].split(")>")[0]
                need_search=True if need_search=="True" else False
                search_word=question_block.split("<Query(")[1].split(")>")[0]
                rewrite_questions.append({
                    "question": question,
                    "needSearch": need_search,
                    "query": search_word
                })
        except:
            print(f"parse questions error: {output_str}")
            rewrite_questions=[]
            
        #  parse claims
        try:
            claims=output_str.split("<Claims>")[1].strip()
            if claims.endswith("</Claims>"):
                claims=claims.split("</Claims>")[0]
            claims=[c for c in claims.split("<Claim(") if c.strip()]
            rewrite_claims=[]
            for claim_block in claims:
                claim_block=claim_block.strip()
                if not claim_block.endswith(")>") or "<NeedSearch(" not in claim_block or "<Query(" not in claim_block:
                    continue
                claim=claim_block.split(")>")[0]
                need_search=claim_block.split("<NeedSearch(")[1].split(")>")[0]
                need_search=True if need_search=="True" else False
                query=claim_block.split("<Query(")[1].split(")>")[0]
                rewrite_claims.append({
                    "claim": claim,
                    "needSearch": need_search,
                    "query": query
                })
        except:
            print(f"parse claims error: {output_str}")
            rewrite_claims=[]
            
        data_line[f"{sep_model}{chat_model}_rewrite"] = {
            "task": task,
            "timeliness": timeliness,
            "questions": rewrite_questions,
            "claims": rewrite_claims
        }
        
    with open(output_file, "w", encoding="utf-8") as f:
        for line in data_lines:
            f.write(json.dumps(line, ensure_ascii=False) + "\n")
    
    

In [ ]:
#  parse llama judge output strings 
from json import JSONDecodeError
# datasets=("asqa", "hotpot-qa", "nq", "trivia-qa", "2wiki", "qampari", "musique", "dolly", "eli5")
datasets=("asqa", "nq", "trivia-qa", "musique", "eli5")
split='test'
rewrite_model='v0104qwen7b'
for dataset in datasets:
    if dataset.startswith("v"):
        llama_rewrite_file=f'../user_intent_data/mixed/{dataset}/rewrite/{rewrite_model}/{rewrite_model}-{dataset}-{split}.jsonl'
        parsed_llama_rewrite_file=f'../user_intent_data/mixed/{dataset}/rewrite/{rewrite_model}/unparsed-{rewrite_model}-{dataset}-{split}.jsonl'
    else:
        llama_rewrite_file=f'../user_intent_data/{dataset}/rewrite/{rewrite_model}/{rewrite_model}-{dataset}-{split}.jsonl'
        parsed_llama_rewrite_file=f'../user_intent_data/{dataset}/rewrite/{rewrite_model}/unparsed-{rewrite_model}-{dataset}-{split}.jsonl'
    with open(parsed_llama_rewrite_file, "r", encoding="utf-8") as f:
        llama_rewrite_resutls = [json.loads(line) for line in f]
        
    #  example output:
    #  {'generated_text': '<Task(information_extraction)> <Timeliness(True)> <Questions> <Question(What are the options for selecting Subsequent Payment terms for a Canadian mortgage?)> <KeyWords(Canadian mortgage Subsequent payment term options)> <NeedSearch(True)> </Questions> ', 'special_ret': {'tensor': []}}
    for idx, data_line in enumerate(llama_rewrite_resutls):
        output_str = data_line[f"{rewrite_model}_rewrite"]
        try:
            known=output_str.split("<Known(")[1].split(")>")[0]
            known=True if known=="True" else False
        except:
            print(f"parse known error: {output_str}")
            known=False
            
        llama_rewrite_resutls[idx][f"{rewrite_model}_rewrite"] = {
            "known": known,
        }
        
            
    with open(llama_rewrite_file, "w", encoding="utf-8") as f:
        for line in llama_rewrite_resutls:
            f.write(json.dumps(line, ensure_ascii=False) + "\n")